In [2]:
print('ok')

ok


In [3]:
%pwd

'/Users/rahulkulkarni/Downloads/Aaryan/Medical-Chatbot/research'

In [4]:
# Going one directory back so as to be able to load data
import os
os.chdir("../")

In [5]:
%pwd

'/Users/rahulkulkarni/Downloads/Aaryan/Medical-Chatbot'

In [6]:
## Document Loader
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader

def load_documents(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents


documents = load_documents("data")
print("Document loaded : ",documents[:2])
print("Total number of documents loaded : ",len(documents))



/Users/rahulkulkarni/Downloads/Aaryan/Medical-Chatbot/venv_medical/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Document loaded :  [Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data/Medical_book.pdf', 'total_pages': 637, 'page': 0, 'page_label': '1'}, page_content=''), Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data/Medical_book.pdf', 'total_pages': 637, 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION')]
Total number of documents loaded :  637


In [7]:
documents[0]

Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data/Medical_book.pdf', 'total_pages': 637, 'page': 0, 'page_label': '1'}, page_content='')

In [11]:
# Filter the document content
from typing import List
from langchain.docstore.document import Document
        
def load_minimal_docs(documents: List[Document]) -> List[Document]:
    minimal_docs = []
    for doc in documents:
        src = doc.metadata.get("source")
        minimal_doc = Document(page_content=doc.page_content, metadata={"source": src})
        minimal_docs.append(minimal_doc)

    return minimal_docs

filtered_docs = load_minimal_docs(documents)
print(filtered_docs[:2])





ModuleNotFoundError: No module named 'langchain.docstore'

In [11]:
## Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
def text_split(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    docs = text_splitter.split_documents(documents)

    return docs

text_chunks = text_split(filtered_docs)
print(len(text_chunks))

5859


In [12]:
## Embedding model to generate embeddings of the text chunks
from langchain.embeddings import HuggingFaceEmbeddings

def download_embedding_model():
    """Download the embedding model from HuggingFace."""
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")   # Has a 384-dimensional embedding space)
    return embedding_model

embedding = download_embedding_model()
print("Embedding model downloaded")


/var/folders/vn/sxr92rkd34zfhswqf9vhz_jm0000gn/T/ipykernel_90785/527128466.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")   # Has a 384-dimensional embedding space)


Embedding model downloaded


In [13]:
vectors = embedding.embed_query('Hello world')
print(len(vectors))

384


In [14]:
from dotenv import load_dotenv
import os
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPEAI_API_KEY"] = OPENAI_API_KEY

In [15]:
## Create Vector Store

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot"
existing_indexes = [idx["name"] for idx in pc.list_indexes()]
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,  # Dimension of the embedding model
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-east-1'),
    )

index = pc.Index(index_name)

In [16]:
# Load the records in pinecone
from langchain_pinecone import PineconeVectorStore
vectorStore = PineconeVectorStore.from_documents(
    documents = text_chunks,
    embedding = embedding,
    index_name = index_name
)

In [17]:
type(vectorStore)

langchain_pinecone.vectorstores.PineconeVectorStore

In [18]:
vectorStore.similarity_search("What is hypertension?", k=2)


[Document(id='4472433f-5bc2-4988-a5f1-1f334027adf5', metadata={'source': 'data/Medical_book.pdf'}, page_content='(BPH), a condition that affects men and is characterized\nby an enlarged prostate gland.\nHigh blood pressure\nHigh blood pressure puts a strain on the heart and\nthe arteries. Over time, hypertension can damage the\nblood vessels to the point of causing stroke, heart fail-\nure or kidney failure. People with high blood pressure\nmay also be at higher risk for heart attacks. Controlling\nhigh blood pressure makes these problems less likely.\nAlpha blockers help lower blood pressure by causing'),
 Document(id='18144051-c21a-4ffb-8889-7a202edb9c27', metadata={'source': 'data/Medical_book.pdf'}, page_content='(BPH), a condition that affects men and is characterized\nby an enlarged prostate gland.\nHigh blood pressure\nHigh blood pressure puts a strain on the heart and\nthe arteries. Over time, hypertension can damage the\nblood vessels to the point of causing stroke, heart fail

In [19]:
## Retriever

retriever = vectorStore.as_retriever(search_type='mmr', lambda_mult=0.5, k=3)

retriever_docs = retriever.invoke("What is hypertension?")
print(retriever_docs)

[Document(metadata={'source': 'data/Medical_book.pdf'}, page_content='(BPH), a condition that affects men and is characterized\nby an enlarged prostate gland.\nHigh blood pressure\nHigh blood pressure puts a strain on the heart and\nthe arteries. Over time, hypertension can damage the\nblood vessels to the point of causing stroke, heart fail-\nure or kidney failure. People with high blood pressure\nmay also be at higher risk for heart attacks. Controlling\nhigh blood pressure makes these problems less likely.\nAlpha blockers help lower blood pressure by causing'), Document(metadata={'source': 'data/Medical_book.pdf'}, page_content='Hypertension—Persistently high arterial blood pres-\nsure.\nNeurotransmitter —Substance released from neu-\nrons of the peripheral nervous system that travels\nacross the synaptic clefts (gaps) of other neurons to\nexcite or inhibit the target cell.\nPalpitation—Rapid, forceful, throbbing, or fluttering\nheartbeat.\nReceptor—A molecular structure in a cell o

In [20]:
def format_docs(docs):
    return "\n".join([d.page_content for d in docs])

docs = format_docs(retriever_docs)
print(docs)

(BPH), a condition that affects men and is characterized
by an enlarged prostate gland.
High blood pressure
High blood pressure puts a strain on the heart and
the arteries. Over time, hypertension can damage the
blood vessels to the point of causing stroke, heart fail-
ure or kidney failure. People with high blood pressure
may also be at higher risk for heart attacks. Controlling
high blood pressure makes these problems less likely.
Alpha blockers help lower blood pressure by causing
Hypertension—Persistently high arterial blood pres-
sure.
Neurotransmitter —Substance released from neu-
rons of the peripheral nervous system that travels
across the synaptic clefts (gaps) of other neurons to
excite or inhibit the target cell.
Palpitation—Rapid, forceful, throbbing, or fluttering
heartbeat.
Receptor—A molecular structure in a cell or on the
surface of a cell that allows binding of a specific
substance that causes a specific physiologic
response.
tain kidney disorders increase the producti

In [23]:
## LLM
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

# llm.invoke('What is hypertension')

In [24]:
## Prompt Template
from langchain.prompts import ChatPromptTemplate

system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "Context:\n{context}"  
)

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}"),
    
])


In [25]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class Medical(BaseModel):
    question: str = Field(description="The medical question to be answered.")
    answer: str = Field(description="The answer to the medical question.")

parser = PydanticOutputParser(pydantic_object=Medical)

In [1]:
from langchain.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

retriever_chain = RunnableParallel({
    'input' : RunnablePassthrough(),
    'context' : retriever | RunnableLambda(format_docs)
})

# prompt template as Runnable
prompt_runnable = RunnableLambda(
    lambda inputs: prompt_template.format_messages(
        input=inputs['input'], 
        context=inputs['context']
    )
)

main_chain = retriever_chain | prompt_template | llm | parser

# result = main_chain.invoke('What is hypertension)
# print(result.summary)





ModuleNotFoundError: No module named 'langchain.runnables'